FAKE NEWS CLASSIFIER </br>
Name: Zuhayer Alvi </br>
CSC 44800

In [ ]:
# Import needed libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [8]:
# load both datasets
df_true = pd.read_csv('datasets/True.csv')
df_fake = pd.read_csv('datasets/Fake.csv')

In [9]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [10]:
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [11]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [12]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [13]:
# insert a column to identify the class of the data
# 1 for true and 0 for fake
df_true['class'] = 1
df_fake['class'] = 0

In [14]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [15]:
df_fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [16]:
# Combine the datasets
merged_df = pd.concat([df_true, df_fake], ignore_index=True)

In [21]:
# Shuffle the dataset
merged_df = merged_df.sample(frac=1).reset_index(drop=True)

In [23]:
merged_df.head(10)

,title,text,subject,date,class
0,Israel willing to resort to military action to...,TOKYO (Reuters) - Israel is willing to resort ...,worldnews,"October 26, 2017",1
1,Vietnam seeks death penalty for embezzlement b...,HANOI (Reuters) - Prosecutors in Vietnam on Th...,worldnews,"September 14, 2017",1
2,“MODERATE” MUSLIM SILENCE? Neighbors: “They Ke...,If the Muslim terrorist who killed 49 people a...,left-news,"Jun 15, 2016",0
3,Turkey cautions citizens about travel to 'anti...,ANKARA (Reuters) - Turkey cautioned its citize...,worldnews,"September 9, 2017",1
4,"School, infrastructure bond measures fill U.S....",CHICAGO/SAN FRANCISCO (Reuters) - U.S. voters ...,politicsNews,"November 4, 2016",1
5,HANDPICKED SUCCESSOR And Son Of DISGRACED Lawm...,The disgraced Representative John Conyers (D-M...,politics,"Dec 6, 2017",0
6,FBI chief says his staff would face discipline...,WASHINGTON (Reuters) - FBI Director James Come...,politicsNews,"July 7, 2016",1
7,Judge Orders Trump To Court In Early January;...,One of Donald Trump s main tools as a business...,News,"December 14, 2016",0
8,TRUMP’S HISTORIC SPEECH: 4 Words That Will Cau...,Trump talked about a country on the brink of d...,politics,"Jul 22, 2016",0
9,REPUBLICANS CALL FOR ANSWERS: Did Wasserman-Sc...,Busted! Even moderate Republican Susan Collins...,politics,"Oct 29, 2017",0


In [ ]:
# Use the 'text' column for modeling
X = merged_df['text']
y = merged_df['class']

In [26]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [33]:
# Build and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [34]:
# Make predictions
y_pred = model.predict(X_test_tfidf)

In [35]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9859688195991091
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4703
           1       0.98      0.99      0.99      4277

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [40]:
# Build and train the Random Forest model
model2 = RandomForestClassifier()
model2.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [41]:
# Make predictions
y_pred = model2.predict(X_test_tfidf)

In [42]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9919821826280624
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4703
           1       0.99      0.99      0.99      4277

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

